In [17]:
#We are going to import from Pandas,Numpy, and sklearn and store the data frame into the variable df [18BIS0103]
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

df = pd.read_csv("IOT-temp dataset task 2.csv")
print (df.head())

                                    id  room_id/id        noted_date  temp  \
0  __export__.temp_log_196134_bd201015  Room Admin  08-12-2018 09:30  29.0   
1  __export__.temp_log_196131_7bca51bc  Room Admin  08-12-2018 09:30  29.0   
2  __export__.temp_log_196127_522915e3  Room Admin  08-12-2018 09:29  41.0   
3  __export__.temp_log_196128_be0919cf  Room Admin  08-12-2018 09:29  41.0   
4  __export__.temp_log_196126_d30b72fb  Room Admin  08-12-2018 09:29  31.0   

  out/in  
0     In  
1     In  
2    Out  
3    Out  
4     In  


In [18]:
#We are going to Drop the coloums that are not required for pridiction [18BIS0103]
#These coloums are id, and room_id/id 

df_dropped = df.drop(columns=["id","room_id/id"])
print(df_dropped.head())

#The coloums show below the coloums droppped

         noted_date  temp out/in
0  08-12-2018 09:30  29.0     In
1  08-12-2018 09:30  29.0     In
2  08-12-2018 09:29  41.0    Out
3  08-12-2018 09:29  41.0    Out
4  08-12-2018 09:29  31.0     In


In [19]:
#These are the values that contain duplicated values
print(df_dropped.shape)

(97606, 3)


In [20]:
#We are going to Remove duplicate values
df_dropped.drop_duplicates(inplace=True)
df_dropped[df_dropped.duplicated()]
entries=df_dropped.shape
entries = int(entries[0])
print(entries)
#We also stored the number of entries for later use
#These are the number of rows after the duplicated values are removed

37281


In [21]:
print(df_dropped.isnull().sum())

#We can see that there are 14 null values in temp coloum. We can put the mean value there.

noted_date     0
temp          14
out/in         0
dtype: int64


In [22]:
#We are going to remove the missing data and put in the MEAN value

missing_value = ['temp']

for column in missing_value:
    df_dropped[column] = df_dropped[column].replace(np.NaN, np.NaN)
    mean = int(df_dropped[column].mean(skipna=True))
    df_dropped[column] = df_dropped[column].replace(np.NaN, mean)
    
print(df_dropped.isnull().sum())

#The below is a proof that the Data is free of null values

noted_date    0
temp          0
out/in        0
dtype: int64


In [23]:
#NOTE : THIS IS THE SAME STEP AS THE PREVIOUS ONE, EXECPT WE USE MEDIAN VALUE
#We are going to remove the missing data and put in the median value

missing_value = ['temp']

for column in missing_value:
    df_dropped[column] = df_dropped[column].replace(np.NaN, np.NaN)
    median = int(df_dropped[column].median(skipna=True))
    df_dropped[column] = df_dropped[column].replace(np.NaN, median)
    
print(df_dropped.isnull().sum())

#The below is a proof that the Data is free of null values

noted_date    0
temp          0
out/in        0
dtype: int64


In [24]:
# Complete the call to convert the date column
df_dropped['noted_date'] =  pd.to_datetime(df_dropped['noted_date'], format='%d-%m-%Y %H:%M')
df_dropped['noted_date'].astype('datetime64[D]').dtype
df_dropped['noted_date'] = pd.to_numeric(pd.to_datetime(df_dropped['noted_date']))
# Confirm the date column is in datetime format
print(df_dropped.head())
print(df_dropped.info())



            noted_date  temp out/in
0  1544261400000000000  29.0     In
2  1544261340000000000  41.0    Out
4  1544261340000000000  31.0     In
6  1544261280000000000  29.0     In
8  1544261160000000000  29.0     In
<class 'pandas.core.frame.DataFrame'>
Int64Index: 37281 entries, 0 to 97603
Data columns (total 3 columns):
noted_date    37281 non-null int64
temp          37281 non-null float64
out/in        37281 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


In [25]:
#Split the dataset into out and in
 
df_dropped_out = df_dropped[df_dropped['out/in'] =='Out'] #This Varialble has OUT in it
df_dropped_in = df_dropped[df_dropped['out/in'] =='In'] #This variable has IN in it

print(df_dropped_out.head())
print(df_dropped_in.head())

             noted_date  temp out/in
2   1544261340000000000  41.0    Out
10  1544261100000000000  42.0    Out
16  1544260860000000000  41.0    Out
20  1544260740000000000  42.0    Out
24  1544260620000000000  41.0    Out
             noted_date  temp out/in
0   1544261400000000000  29.0     In
4   1544261340000000000  31.0     In
6   1544261280000000000  29.0     In
8   1544261160000000000  29.0     In
12  1544261040000000000  29.0     In


In [26]:
#Splitting dataset for training and testing for OUT --> 1 for out, 2 --> IN

X1 = df_dropped_out.iloc[:, 0:2]
y1 = df_dropped_out.iloc[:, 1]
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,random_state=0, test_size=0.2)

In [40]:
#Splitting dataset for training and testing for IN --> 1 for out, 2 for in

X2 = df_dropped_in.iloc[:, 0:2]
y2 = df_dropped_in.iloc[:, 1]
X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y2,random_state=0, test_size=0.2)

In [41]:
#Feature scaling
sc_X1 = StandardScaler()
X1_train = sc_X1.fit_transform(X1_train)
X1_test = sc_X1.transform(X1_test)
sc_X2 = StandardScaler()
X2_train = sc_X2.fit_transform(X2_train)
X2_test = sc_X2.transform(X2_test)

In [42]:
#import math
#math.sqrt(len(y1_test))
classifier_IN = KNeighborsClassifier(n_neighbors= 3, p = 2, metric = 'euclidean')
classifier_IN.fit(X2_train, y2_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [44]:
classifier_OUT = KNeighborsClassifier(n_neighbors= 3, p = 'weighted', metric = 'euclidean')
classifier_OUT.fit(X1_train, y1_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=3, p='weighted',
           weights='uniform')

In [47]:
y1_pred = classifier_OUT.predict(X1_test)
y2_pred = classifier_IN.predict(X2_test)
cm = confusion_matrix (y1_test,y1_pred)
print (cm)
print (f1_score(y1_test,y1_pred, average='weighted'))

[[  2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   4   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0  19   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0 139   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   1 256   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 203   1   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0 239   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 254   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 139   1   0   0   0   0   0   

C:\Users\Anurag Dhudhwani\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [48]:
cm_in = confusion_matrix(y2_test, y2_pred)
print (cm_in)
print (f1_score(y2_test, y2_pred, average='weighted'))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1   1   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   5   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  18   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  40   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 113   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0 156   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 300   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 282   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 331   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 291   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0 232   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0 152   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  60   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   8]]
0.9996649916247907


C:\Users\Anurag Dhudhwani\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
